# <span style='font-family: CMU Sans Serif, sans-serif;'> Recurrent neural networks  </span> 

These models are designed to process timeseries and other sequence data.

Here vanilla RNN's and improved variants such as long short-term memory (LSTM) models.

## <span style='font-family: CMU Sans Serif, sans-serif;'> Vanilla RNN's  </span> 

<div style='border: 1px solid currentColor; border-radius: 2px; padding: 10px; width: 95%;'>

<span style='font-family: CMU Sans Serif, sans-serif; font-weight: bold; font-size: 17px;'>
Recurrent neural networks setup
</span> 

We have time series inputs $\mathbf{x}_1, \mathbf{x}_2, \ldots, \mathbf{x}_{T}$. A vanilla RNN models the hidden state at time $t$ by a vector $\mathbf{h}_{t}$, which is subject ot the recursive formula
$$
\mathbf{h}_t = \mathbf{f}_{\boldsymbol{\theta} }(\mathbf{h}_{t-1}, \mathbf{x}_t)
$$

Here $f_{\boldsymbol{\theta} }$ is generally a non-linear function parametrized by $\boldsymbol{\theta}$. Concretely, a vanilla RNN with one hidden layer has the following form
\begin{align*}
    \mathbf{h}_t & = \mathbf{\tanh}(\mathbf{W}_{hh} \mathbf{h}_{t-1} + \mathbf{W}_{xh}\mathbf{x}_t + \mathbf{b}_{\mathbf{h}}), \quad \quad \text{where} \tanh(a) = \frac{e^{2a} - 1}{e^{2a} + 1} \\
    \mathbf{z}_t & = \boldsymbol{\sigma} (\mathbf{W}_{hy} \mathbf{h}_{t} + \mathbf{b}_{\mathbf{z}})
\end{align*}
where $\mathbf{W}_{hh}$, $\mathbf{W}_{xh}$, $\mathbf{W}_{hy}$ are trainable weights and matrices, $\mathbf{b}_{\mathbf{h}}$, $\mathbf{b}_{\mathbf{z}}$ are trainable bias vectors, and $\mathbf{z}_{t}$ is the output at time $t$. These parameters are shared across time. 
</div>

Below is an example of how a recurrent neural networks looks when unfolded.
<div style='text-align:center;'>
    <img src='figs/rnns/rnn__unfold.svg' width='1000'>
</div>


Below is a plot of the different type of input output settings.
<div style='text-align:center;'>
    <img src='figs/rnns/rnn__in_out__setting.svg' width='1500'>
</div>

We can have different input/output settings. Consider the following: 
- **One-to-many**: a single input with multiple outputs. A typical application is image captioning, where the input is an image and the outputs are a series of words.
- **Many-to-one**: multiple inputs with a single output. One application is text sentiment classification, where the input is a series of words in a sentence, and the output is a label.
- **Many-to-many**: multiple inputs and outputs. This is used in machine translation, where inputs are words of a source language and outputs are words of a taget language. 


As with feed-forward neural networks we minimize the loss function using back propagation, where to loss can be
$$
\ell_T(\boldsymbol{\theta} ) = \sum_{t \in \mathcal{T}} \mathcal{L}(y_t, \mathbf{z}_t) = - \sum_{t \in \mathcal{T}} \sum_{k=1}^{K} I(y_t = k) \log \left( \frac{\exp([\mathbf{z}_t]_k)}{\sum_k \exp([\mathbf{z}_t]_k)} \right) 
$$
where $K$ is the number of categories for classification, and $\mathcal{T} \subset [T] $ is the length of the output sequence. During training, the gradients $\frac{\partial \ell_{\mathcal{T}}}{\partial \mathbf{h}_t}$ are computed in the reverse time order (from $T$ to $t$). For this reason, the training process is often called **back-propagation trough time**.



One big drawback of recurrent neural networks is that they have a hard time capturing long-range dependencies in sequence data when the length of the sequence is large. This is often called **exploding/vanishing gradients**. 

Consider the many-to-many graph shown above. Calculating $\frac{\partial \ell_{\mathcal{T}}}{\partial \mathbf{h}_1}$ involves the product $\prod_{t=1}^{3} (\frac{\partial \mathbf{h}_{t+1}}{\partial \mathbf{h}_t})$ by the chain rule. If this sequence is long then we have to multiply many Jacobian matrices together, which often results in very large or small single values.

To alleviate this issue the forward pass and backward pass are implemented in a shorter sliding window $\{ t_1, t_1+1, t_1+2, \ldots ,t_2 \}$ instad of the full sequence $\{ 1,2, \ldots , T \}$. Whilst effective, this does not always fully address the long-term dependencies.

## <span style='font-family: CMU Sans Serif, sans-serif;'>  GRUs and LSTM </span> 

<div style='border: 1px solid currentColor; border-radius: 2px; padding: 10px; width: 95%;'>

<span style='font-family: CMU Sans Serif, sans-serif; font-weight: bold; font-size: 17px;'>
GRU & LSTM
</span> 

These are improved variants to alleviate the issue mentioned above: gated recurrent units (GRUs) and long short-term memory (LSTM). 
- A **GRU** refines the recursive formula by introducing gates, which are vectors of the same length as $\mathbf{h}_t$. The gates, that take values in $[0,1]$ elementwise, multiply with $\mathbf{h}_{t-1}$ elementwise and determine how much the keep the old hidden states.
- An **LSTM** similarly ueses gates in the recursive formula. In addition to $\mathbf{h}_t$, an LSTM maintains a cell state, which takes values in $\mathbb{R}$ elementwise and are analogous to counters. 

Denote by $\odot$ the element-wise multiplication. We have a recursive formula substituting the first recursive formula given in the first RRN: 

\begin{gather*}
    \begin{pmatrix}
    \boldsymbol{i}_t \\
    \boldsymbol{f}_t \\
    \boldsymbol{o}_t \\
    \boldsymbol{g}_t
\end{pmatrix}
= 
\begin{pmatrix}
    \boldsymbol{\sigma} \\
    \boldsymbol{\sigma} \\
    \boldsymbol{\sigma} \\
    \boldsymbol{\tanh} 
\end{pmatrix}
\mathbf{W} 
\begin{pmatrix}
    \mathbf{h}_{t-1} \\
    \mathbf{x}_t \\
    1
\end{pmatrix} \\
\mathbf{c}_t = \mathbf{f}_t \odot \mathbf{c}_{t-1} + \boldsymbol{i}_t \odot \mathbf{g}_t  \\
\mathbf{h}_t = \mathbf{o}_t \odot \boldsymbol{\tanh}(\mathbf{c}_t),
\end{gather*}

where $\mathbf{W}$ is a weight matrix with appropriate dimensions. The cell state vector $\mathbf{c}_t$ carries information of the sequence. The forget gate $\mathbf{f}_t$ determines how much the values of $\mathbf{c}_{t-1}$ are kept for time $t$, the input gate $\boldsymbol{i}_t$ controls the amount of update to the cell state, and the output gate $\mathbf{o}_t$ gives how much $\mathbf{c}_t$ reveals to $\mathbf{h}_t$. Ideally, the elements of these gates have nearly binary values. Consider, if an element of $\mathbf{f}_t$ is close to $1$, then it might suggest the presence of a feature in the sequence data. Similar to the skip connections in residual nets, the cell state $\mathbf{c}_t$ has an additive recursive formula, which helps back-propagation and thus captures long-range dependencies.

</div>


## <span style='font-family: CMU Sans Serif, sans-serif;'> Multilayer RNNs  </span> 

Generalization of one-hidden-layer RNN (above). In place of $\mathbf{h}_t = \mathbf{f}_{\boldsymbol{\theta}}(\mathbf{h}_{t-1}, \mathbf{x}_t)$ multilayer recurrent neural networks with $L$ layers use
$$
\mathbf{h}_t^{\ell} = \boldsymbol{\tanh} \left[ \mathbf{W}^{\ell}  \begin{pmatrix} \mathbf{h}_t^{\ell-1} \\ \mathbf{h}_{t-1}^{\ell} \\ 1 \end{pmatrix} \right], \quad \forall \, \ell \in [L], \quad \mathbf{h}_t^{0} \triangleq \mathbf{x}_t
$$

Note these types of neural networks have two dimensions $L$ which is the dept and $L$ which is the sequence length.  Two special cases; feed-forward neural nets ($T=1$) and RNN's with one hidden layer ($L = 1$). 

Below is an illustration of multilayer recurrent neural network.

<div style='text-align:center;'>
    <img src='figs/rnns/ml__rnn.svg' width='650'>
</div>

These types of neural networks usually do not have very large depth (e.g., $2$ to $5$) since $T$ is already very large.

### <span style='font-family: CMU Sans Serif, sans-serif;'> Combination of neural networks  </span> 

CNNs, RNNs, and other neural networks can be combined to tackle tasks that involve different sources of input data. This fact provides a lot of flexibility in various applications. 